In [ ]:
from FinMind.data import DataLoader
from api_util import get_api_key

api_key = get_api_key("FIN_MIND_KEY")

fin_mind = DataLoader()
fin_mind.login_by_token(api_token=api_key)

In [ ]:
stock_info = fin_mind.taiwan_stock_info()

In [ ]:
print(stock_info.columns)
result = stock_info[stock_info['stock_name'].str.contains('長榮航')]
print(result)

In [ ]:

import pandas as pd
df = fin_mind.taiwan_stock_daily(
    stock_id='3306',
    start_date='2023-01-01',
    end_date='2023-06-03'
)

# original date column has type object, convert it to datetime64[ns]
df['date'] = pd.to_datetime(df['date'])

# Set the 'date' column as the DataFrame index
df.set_index('date', inplace=True)

print(df.columns)

In [ ]:
import ta

from ta.volatility import BollingerBands
from ta.momentum import rsi

bb_band_window = 60
bb_band_window_dev=2

# Initialize Bollinger Bands Indicator
indicator_bb = BollingerBands(close=df["close"],
                              n=bb_band_window,
                              ndev=bb_band_window_dev)

# Add Bollinger Bands features
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()
df['bb_diff'] = df['bb_bbh'] - df['bb_bbl']
df['bb_ratio'] = df['bb_diff'] / df['bb_diff'].shift(1)


# Add Bollinger Band high indicator
#df['bb_bbhi'] = indicator_bb.bollinger_hband_indicator()

# Add Bollinger Band low indicator
#df['bb_bbli'] = indicator_bb.bollinger_lband_indicator()


# Calculate Relative Strength Index (RSI)
for i in range(2, 10):
    df[f"{i}-rsi"] = rsi(df['close'], n=i, fillna=True)

df['rsi_6week'] = rsi(df['close'], n=42, fillna=True)
df['rsi_6month'] = rsi(df['close'], n=180, fillna=True)

# Calculate average volume for different periods
df['avg_volume_4'] = df['Trading_Volume'].rolling(window=4).mean()


In [ ]:
print(df.tail(3))

In [ ]:
import numpy as np

def calculate_bbands(df, close_col, period, num_std_devs):
    # Calculate the Simple Moving Average (SMA) using a rolling window
    sma = df[close_col].rolling(window=period).mean()

    # Calculate the standard deviation using a rolling window
    std_dev = df[close_col].rolling(window=period).std()

    # Calculate the upper and lower bands
    upper_band = sma + (std_dev * num_std_devs)
    lower_band = sma - (std_dev * num_std_devs)

    return upper_band, lower_band


def calculate_rsi(df, closing_price_col, period):
    closing_prices = df[closing_price_col].values
    price_changes = np.diff(closing_prices)
    gains = np.where(price_changes > 0, price_changes, 0)
    losses = np.where(price_changes < 0, abs(price_changes), 0)

    avg_gain = np.mean(gains[:period])
    avg_loss = np.mean(losses[:period])
    
    rs = avg_gain / avg_loss if avg_loss != 0 else 0
    
    rsi = 100 - (100 / (1 + rs))
    
    return rsi






ub, lb = calculate_bbands(df, 'close', 20, 2)
print("-------------")
print(ub)
#print(lb)